# Estimate Tests

In [1]:
import os 
import pandas as pd
import numpy as np
import glob
import pyodbc
import gc

SQL Data

In [3]:
conn = pyodbc.connect('Driver={ODBC Driver 17 for SQL Server};'
                    'Server=DDAMWSQL16.sandag.org;'
                    'Database=estimates;'
                    'Trusted_Connection=yes;')

In [4]:
age_query = '''SELECT [mgra_id]
      ,[yr_id]
      ,[age_group_id]
      ,[population]
  FROM [estimates].[est_2021_01].[dw_age]'''

age_df =  pd.read_sql_query(age_query, conn) #example df

Dummy data

In [15]:
df = pd.read_csv('data/2020_06_consolidated_cpa_QA.csv')
df.head()

,cpa,yr_id,Under 5,5 to 9,10 to 14,15 to 17,18 and 19,20 to 24,25 to 29,30 to 34,...,Mobile Home,Single-family Detached,Single-family Attached,Total Population,Household Population,Group Quarters - Military,Group Quarters - College,Group Quarters - Other,Female,Male
0,*Not in a CPA*,2010,90871,88551,91299,58584,38383,100872,95918,87962,...,23408,0,0,1307092,1278994,12565,757,14776,662153,644939
1,*Not in a CPA*,2011,92624,89970,91020,58625,37208,102829,95803,89077,...,23521,0,0,1319085,1292819,11006,732,14528,666987,652098
2,*Not in a CPA*,2012,94265,92503,90614,59753,37398,105358,94585,90851,...,23579,0,0,1334407,1308219,10688,747,14753,674166,660241
3,*Not in a CPA*,2013,95707,95149,90674,60809,38364,107063,93448,92897,...,23679,0,0,1352612,1325969,10670,1003,14970,681992,670620
4,*Not in a CPA*,2014,96320,97534,91130,60475,39052,107246,92293,94577,...,23742,0,0,1365946,1338945,10777,1184,15040,688018,677928


# Checking for null values

If using sql query:

In [5]:
age_query = '''SELECT [mgra_id]
      ,[yr_id]
      ,[age_group_id]
      ,[population]
  FROM [estimates].[est_2021_01].[dw_age]
  WHERE [mgra_id] is NULL'''

age_df =  pd.read_sql_query(age_query, conn)

In [6]:
age_df # no null vals so empty

,mgra_id,yr_id,age_group_id,population


If using a df:

In [16]:
def spot_nulls(df):
    if (df.isna().sum() != 0).any():
        print('Null values present')
        return df.columns[df.isnull().any()].tolist() # returns the column names that have null vals

In [17]:
spot_nulls(df)

Code for Y-o-Y change using outputs saved by Eric as dummy (Prabina)
Threshold Analysis- Calculate Year-on-Year % changes and flag  changes >5%

# Year-on-Year % change

In [30]:
def yearly_change(df, col, region_level):
    df = df.loc[:,~df.columns.isin([region_level])] 
    pop_change = df.groupby('yr_id').sum().pct_change()[[col]]
    return pop_change[pop_change[col]>0.05].index.to_list() # returns years that have changes more than 5%

In [31]:
yearly_change(df, 'Total Population', 'cpa')

[]